In [ ]:
import zipfile
import io
import requests
import pandas as pd
import os
from google.colab import drive


In [ ]:

drive.mount('/content/drive/MyDrive/PROVEDORES_SP')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
interest_files = [
    "Acessos_Banda_Larga_Fixa_2021.csv",
    "Acessos_Banda_Larga_Fixa_2022.csv",
    "Acessos_Banda_Larga_Fixa_2023.csv",
    "Acessos_Banda_Larga_Fixa_2024.csv",
]

In [ ]:
url = "https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_banda_larga_fixa.zip"
extract_folder = "raw"

response = requests.get(url)
zip_content = io.BytesIO(response.content)

with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    for arquivo in zip_ref.namelist():
        if arquivo in interest_files:
            zip_ref.extract(arquivo, extract_folder)
            print(f"Arquivo extraído: {arquivo}")


In [ ]:
df_list = []
for i in range(1, 5):
    var_name = f"df_vinte_{i}"
    globals()[var_name] = pd.read_csv(f"raw/Acessos_Banda_Larga_Fixa_202{i}.csv", sep=";")
    df_list.append(globals()[var_name])

In [ ]:
def drop_null(df):
    df.dropna(inplace=True)
    return df

In [ ]:
for i in range(len(df_list)):
  df_list[i] = drop_null(df_list[i])

In [ ]:
df_vinte = pd.concat(df_list, ignore_index=True)

df_sp = df_vinte[df_vinte['UF'] == 'SP']
df_sp = df_sp.drop(columns=['UF', 'CNPJ'])

In [ ]:
df_sp = df_sp.rename(columns={'Ano': 'ano',
                              'Mês': 'mes',
                              'Grupo Econômico':
                              'grupo_economico',
                              'Empresa': 'empresa',
                              'Porte da Prestadora': 'porte_prestadora',
                              'Município': 'municipio',
                              'Código IBGE Município': 'cod.municipio',
                              'Faixa de Velocidade': 'faixa_velocidade',
                              'Velocidade': 'velocidade',
                              'Tecnologia': 'tecnologia',
                              'Meio de Acesso': 'meio_acesso',
                              'Tipo de Pessoa': 'tipo_pessoa',
                              'Tipo de Produto': 'tipo_produto',
                              'Acessos': 'acessos'})

In [ ]:
df_sp.velocidade = df_sp.velocidade.str.replace(',', '.').astype(float)
df_sp['empresa'] = df_sp['empresa'].str.upper().str.strip()
df_sp['grupo_economico'] = df_sp['grupo_economico'].str.upper().str.strip()


In [ ]:
output_path = '/content/drive/MyDrive/PROVEDORES_SP/processed/Acessos_Banda_Larga_Fixa_2021-2024_Exportado.csv'
df_sp.to_csv(output_path, index=False)